<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Construcción-de-módulos" data-toc-modified-id="Construcción-de-módulos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Construcción de módulos</a></span></li><li><span><a href="#Librerías" data-toc-modified-id="Librerías-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Librerías</a></span><ul class="toc-item"><li><span><a href="#Ejemplo-1---hello.py" data-toc-modified-id="Ejemplo-1---hello.py-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Ejemplo 1 - hello.py</a></span></li><li><span><a href="#Cargar-datos" data-toc-modified-id="Cargar-datos-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Cargar datos</a></span></li><li><span><a href="#Calcular-drawdowns" data-toc-modified-id="Calcular-drawdowns-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Calcular drawdowns</a></span></li><li><span><a href="#Visualización-completa" data-toc-modified-id="Visualización-completa-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Visualización completa</a></span></li><li><span><a href="#Visualización-de-un-tramo-temporal" data-toc-modified-id="Visualización-de-un-tramo-temporal-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Visualización de un tramo temporal</a></span></li></ul></li></ul></div>

## Construcción de módulos

Un módulo en Python es un archivo con código python al que se puede acceder y ejecutar importando el módulo. Cuando se importa un módulo, se accede al archivo para leer el código que contiene a través de un alias (opcional).

## Librerías

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
import warnings
import numpy as np


# Configuración de librerías
init_notebook_mode(connected=True)  
warnings.filterwarnings("ignore")

### Ejemplo 1 - hello.py

In [1]:
import modules.hello as h

In [3]:
h.message1

'Hello Sergio'

Ahora, cambiamos el código de hello.py añadiendo un segundo mensaje.

```python
message2 = '¿Como estas?'
```

In [4]:
import modules.hello as h
h.message2

AttributeError: module 'modules.hello' has no attribute 'message2'

¿Qué ha fallado?

Dado que el contenido de los módulos no suele cambiar mucho (excepto durante el desarrollo del módulo) Python es inteligente a la hora de importar módulos y una vez que se importa un módulo, no se molesta en reimportarlo. Normalmente, esto es lo que quieres, pero en nuestro caso, cuando estamos desarrollando el módulo, queremos forzar que el módulo se recargue automáticamente cada vez que cambie.

Afortunadamente, hay una secuencia de comandos _mágica_ que hace exactamente eso. Primero, tenemos que cargar una extensión llamada `autoreload` ejecutando el _comando mágico_ `%load_ext autoreload`. Una vez cargada esa extensión, ahora tenemos acceso a un nuevo _comando mágico_ llamado `%autoreload` que admite diferentes modos de autocarga. El modo que queremos por ahora es recargar automáticamente cualquier cosa que cambie, que es el modo 2. Por lo tanto, es necesario ejecutar:

```python
%load_ext autoreload
%autoreload 2
```

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import modules.hello as h
print(h.message1)
print(h.message2)

Hola Sergio
¿Cómo estas?


Ahora, vamos a crear nuestro nuevo módulo, en el que nos basaremos a lo largo del curso, y poner la función drawdown que hemos creado.

In [24]:
from modules import edhec_risk_kit as erk

### Cargar datos

In [39]:
df_returns = erk.get_ffme_returns(directory='../data')
df_returns.head()

SmallCap  LargeCap
1926-07-01   -0.0145    0.0329
1926-08-01    0.0512    0.0370
1926-09-01    0.0093    0.0067
1926-10-01   -0.0484   -0.0243
1926-11-01   -0.0078    0.0270

### Calcular drawdowns

In [51]:
df_drawdowns_sc = erk.drawdown(stock_returns=df_returns['SmallCap'], capital_inicial=1000)
df_drawdowns_sc.head()

Wealth_Index  Previous_Peaks  Drawdown
1926-07-01    985.500000      985.500000  0.000000
1926-08-01   1035.957600     1035.957600  0.000000
1926-09-01   1045.592006     1045.592006  0.000000
1926-10-01    994.985353     1045.592006 -0.048400
1926-11-01    987.224467     1045.592006 -0.055822

In [52]:
df_drawdowns_lc = erk.drawdown(stock_returns=df_returns['LargeCap'], capital_inicial=1000)
df_drawdowns_lc.head()

Wealth_Index  Previous_Peaks  Drawdown
1926-07-01   1032.900000     1032.900000    0.0000
1926-08-01   1071.117300     1071.117300    0.0000
1926-09-01   1078.293786     1078.293786    0.0000
1926-10-01   1052.091247     1078.293786   -0.0243
1926-11-01   1080.497711     1080.497711    0.0000

In [56]:
# Calcular Max Drawdown
sc_max_drawdown, sc_date_max_drawdown = erk.max_drawdown(df_drawdowns_sc["Drawdown"])
lc_max_drawdown, lc_date_max_drawdown = erk.max_drawdown(df_drawdowns_lc["Drawdown"])

print('Max Drawdown:')
print(f'\t- Small Cap = {sc_max_drawdown}. Date: {sc_date_max_drawdown}')
print(f'\t- Large Cap = {lc_max_drawdown}. Date: {lc_date_max_drawdown}')

Max Drawdown:
	- Small Cap = -0.8330007793945303. Date: 1932-05
	- Large Cap = -0.8400375277943123. Date: 1932-05


### Visualización completa

In [53]:
erk.plot_drawdowns(df_drawdowns_sc, title='DrawDowns Small Capital')

In [54]:
erk.plot_drawdowns(df_drawdowns_lc, title='DrawDowns Large Capital')

### Visualización de un tramo temporal

In [58]:
max_year='1950'
erk.plot_drawdowns(df_drawdowns_sc.loc[:max_year], title='DrawDowns Small Capital')

In [60]:
max_year='1950'
erk.plot_drawdowns(df_drawdowns_lc.loc[:max_year], title='DrawDowns Large Capital')